# Import Libraries

In [ ]:
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import joblib



# Single Feature

# Feature Extraction and training - testing

In [ ]:
# a function to extract the STFT features from each audio file and store them in a feature matrix:

def extract_features(folder, n_fft=2048, hop_length=512):
    features = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        y, sr = librosa.load(filepath, sr=None) # Load the audio file using its native sampling rate
        stft = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length, win_length=n_fft)) # Compute the STFT
        features.append(stft.flatten()) # Flatten the spectrogram and add it to the feature matrix
    return np.array(features)


In [ ]:
folder1 = '/home/user/Downloads/drone'
folder2 = '/home/user/Downloads/noise'
# Extract features for both folders
features1 = extract_features(folder1)
features2 = extract_features(folder2)

# Create the labels for the two classes
labels1 = np.ones(features1.shape[0])
labels2 = np.zeros(features2.shape[0])

# Combine the features and labels into a single matrix
features = np.vstack((features1, features2))
labels = np.concatenate((labels1, labels2))


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


# linear_svm

In [ ]:
# Train an SVM classifier
linear_svm = SVC(kernel='linear', C=1, gamma='scale')
linear_svm.fit(X_train, y_train)


SVC(C=1, kernel='linear')

In [ ]:
# Use the trained classifier to predict the labels of the test set
linear_y_pred = linear_svm.predict(X_test)


In [ ]:
# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, linear_y_pred)
print(conf_matrix)


[[179  24]
 [ 10 189]]


In [ ]:
# Calculate the accuracy
acc = accuracy_score(y_test, linear_y_pred)
print("Accuracy:", acc)

# Calculate the precision score
prec = precision_score(y_test, linear_y_pred)
print("Precision:", prec)

# Calculate the recall score
rec = recall_score(y_test, linear_y_pred)
print("Recall:", rec)

# Calculate the f1 score
f1 = f1_score(y_test, linear_y_pred)
print("F1 score:", f1)

# Generate a classification report
report = classification_report(y_test, linear_y_pred)
print("Classification Report:\n", report)


Accuracy: 0.9154228855721394
Precision: 0.8873239436619719
Recall: 0.949748743718593
F1 score: 0.9174757281553397
Classification Report:
               precision    recall  f1-score   support

         0.0       0.95      0.88      0.91       203
         1.0       0.89      0.95      0.92       199

    accuracy                           0.92       402
   macro avg       0.92      0.92      0.92       402
weighted avg       0.92      0.92      0.92       402



In [ ]:
# Save the linear kernel SVM model to a file
with open('linear_svm_model_singlefeature_matrixclassfifcation.pkl', 'wb') as f:
    pickle.dump(linear_svm, f)


# Cubic kernal

In [ ]:
# Assuming X_train, y_train, X_test, and y_test contain the training and testing data, respectively
cubic_svm_model = SVC(kernel='poly', degree=3)

# Train the model on the training data
cubic_svm_model.fit(X_train, y_train)

# Make predictions on the testing data
cubic_y_pred = cubic_svm_model.predict(X_test)

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, cubic_y_pred)
print("Confusion Matrix:\n", conf_matrix)


Confusion Matrix:
 [[190  13]
 [ 97 102]]


In [ ]:
# Calculate the accuracy
acc = accuracy_score(y_test, cubic_y_pred)
print("Accuracy:", acc)

# Calculate the precision score
prec = precision_score(y_test, cubic_y_pred)
print("Precision:", prec)

# Calculate the recall score
rec = recall_score(y_test, cubic_y_pred)
print("Recall:", rec)

# Calculate the f1 score
f1 = f1_score(y_test, cubic_y_pred)
print("F1 score:", f1)

# Generate a classification report
report = classification_report(y_test, cubic_y_pred)
print("Classification Report:\n", report)


Accuracy: 0.7263681592039801
Precision: 0.8869565217391304
Recall: 0.5125628140703518
F1 score: 0.6496815286624203
Classification Report:
               precision    recall  f1-score   support

         0.0       0.66      0.94      0.78       203
         1.0       0.89      0.51      0.65       199

    accuracy                           0.73       402
   macro avg       0.77      0.72      0.71       402
weighted avg       0.77      0.73      0.71       402



In [ ]:
# Save the cubic kernel SVM model to a file
with open('cubic_svm_model_singlefeature_matrixclassfifcation.pkl', 'wb') as f:
    pickle.dump(cubic_svm_model, f)


# Finding Best Parameter and run svm based on that parameter

In [ ]:
# Define the hyperparameter search space
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto'],
}

# Create an SVM classifier
svm_model = SVC()

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and corresponding performance
print('Best hyperparameters:', grid_search.best_params_)
print('Best validation accuracy: {:.2f}%'.format(grid_search.best_score_ * 100))


Best hyperparameters: {'C': 10, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Best validation accuracy: 95.77%


In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report
import joblib
import pickle

# Define the SVM model with the best hyperparameters
svm_model = svm.SVC(C=10, kernel='rbf', degree=2, gamma='scale')

# Train the model on your training data
svm_model.fit(X_train, y_train)

# Make predictions on your test data
y_pred = svm_model.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Save the trained model to disk as a pkl file
with open('best_hyp_svm_model_singlefeature_matrixclassfifcation.pkl', 'wb') as f:
    pickle.dump(svm_model, f)


Accuracy: 97.51%
Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98       203
         1.0       0.97      0.97      0.97       199

    accuracy                           0.98       402
   macro avg       0.98      0.98      0.98       402
weighted avg       0.98      0.98      0.98       402



# Knn

In [ ]:
# Define the k-NN model
knn = KNeighborsClassifier(n_neighbors=5)

# Train the k-NN model on the training data
knn.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = knn.predict(X_test)

In [ ]:
# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Generate a classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)

Accuracy: 0.9477611940298507
Confusion Matrix:
 [[189  14]
 [  7 192]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.96      0.93      0.95       203
         1.0       0.93      0.96      0.95       199

    accuracy                           0.95       402
   macro avg       0.95      0.95      0.95       402
weighted avg       0.95      0.95      0.95       402



In [ ]:
# Save the trained k-NN model using joblib
joblib.dump(knn, 'knn_model_singlefeature_matrixclassfifcation.pkl')

['knn_model_singlefeature_matrixclassfifcation.pkl']

# Load Model 

In [ ]:
# # Load the linear kernel SVM model from the file
# with open('linear_svm_model.pkl', 'rb') as f:
#     linear_svm_model = pickle.load(f)

# # Load the cubic kernel SVM model from the file
# with open('cubic_svm_model.pkl', 'rb') as f:
#     cubic_svm_model = pickle.load(f)


# **Multiple Features**

# Multiple Features Extraction and test_train 

In [ ]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

# Define the paths to the audio files
drone_present_folder = '/home/user/Downloads/drone'
drone_not_present_folder = '/home/user/Downloads/noise'

# Define a function to extract the features
def extract_features(folder):
    features = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        y, sr = librosa.load(filepath, sr=None) # Load the audio file
        psd = np.abs(np.fft.rfft(y)) ** 2 # Compute the PSD
        rms_psd = np.sqrt(psd / len(y)) # Compute the RMS PSD
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13) # Compute the MFCC
        feature = np.concatenate([psd, rms_psd, mfcc.mean(axis=1)]) # Concatenate the features
        features.append(feature) # Add the feature to the feature matrix
    return np.array(features)

# Extract the features for drone present and drone not present audio files
dp_features = extract_features(drone_present_folder)
dnp_features = extract_features(drone_not_present_folder)

# Create the labels for the data
dp_labels = np.ones(dp_features.shape[0])
dnp_labels = np.zeros(dnp_features.shape[0])

# Concatenate the features and labels
X = np.concatenate([dp_features, dnp_features])
y = np.concatenate([dp_labels, dnp_labels])
X = np.nan_to_num(X)
# Check if the features array is empty
if X.shape[0] == 0:
    raise ValueError("Features array is empty.")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



# Svm Best hyp

In [ ]:
# Train the SVM model
svm_model = SVC(kernel='rbf')
svm_params = {'C': [0.1, 1, 10], 'gamma': [0.001, 0.01, 0.1]}
svm_gs = GridSearchCV(svm_model, svm_params, cv=5, n_jobs=-1)
svm_gs.fit(X_train, y_train)

# Save the SVM model
joblib.dump(svm_gs.best_estimator_, 'rbf_svm_model_multiplefeature_matrix_classification.pkl')

# Predict on the test set using the SVM model
svm_y_pred = svm_gs.predict(X_test)

# Calculate the accuracy of the SVM model
svm_accuracy = accuracy_score(y_test, svm_y_pred)

# Calculate the classification report of the SVM model
svm_classification_report = classification_report(y_test, svm_y_pred)

# Calculate the confusion matrix of the SVM model
svm_confusion_matrix = confusion_matrix(y_test, svm_y_pred)


print("SVM Model Results:")
print("Best hyperparameters: ", svm_gs.best_params_)
print("Best validation accuracy: {:.2f}%".format(svm_gs.best_score_ * 100))
print("Accuracy on test set: {:.2f}%".format(svm_accuracy * 100))
print("Classification report:\n", svm_classification_report)
print("Confusion matrix:\n", svm_confusion_matrix)
print('SVM accuracy:', svm_accuracy)

SVM Model Results:
Best hyperparameters:  {'C': 10, 'gamma': 0.001}
Best validation accuracy: 85.93%
Accuracy on test set: 86.07%
Classification report:
               precision    recall  f1-score   support

         0.0       0.80      0.96      0.87       203
         1.0       0.94      0.76      0.84       199

    accuracy                           0.86       402
   macro avg       0.87      0.86      0.86       402
weighted avg       0.87      0.86      0.86       402

Confusion matrix:
 [[194   9]
 [ 47 152]]
SVM accuracy: 0.8606965174129353


# Svm Cubic

In [ ]:
from sklearn import svm
import pickle

# Define the cubic SVM model
svm_model = svm.SVC(kernel='poly', degree=3, coef0=0.0, C=1.0)

# Train the model on your training data
svm_model.fit(X_train, y_train)

# Make predictions on your test data
y_pred = svm_model.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("SVM accuracy:", accuracy)

# Save the trained model to disk as a pkl file
with open('cubic_svm_model_multiplefeature_matrix_classification.pkl', 'wb') as f:
    pickle.dump(svm_model, f)


SVM accuracy: 0.6716417910447762


# Knn

In [ ]:
from sklearn.metrics import accuracy_score
# Train the KNN model
knn_model = KNeighborsClassifier()
knn_params = {'n_neighbors': [3, 5, 7]}
knn_gs = GridSearchCV(knn_model, knn_params, cv=5, n_jobs=-1)
knn_gs.fit(X_train, y_train)

# Save the KNN model
joblib.dump(knn, 'knn_model_multiplefeature_matrix_classification.pkl')

# Train the KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

# Predict the labels of the test set using the KNN model
y_pred_knn = knn_model.predict(X_test)

# Calculate the accuracy of the KNN model
knn_accuracy = accuracy_score(y_test, y_pred_knn)
print("KNN accuracy:", knn_accuracy)

KNN accuracy: 0.9129353233830846


# Best hyp parameter with too many parameter to find best accuracy

In [ ]:
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'gamma': [0.001, 0.01, 0.1, 'scale', 'auto'],
    'degree': [2, 3, 4, 5],
    'coef0': [0.0, 0.5, 1.0]
}

# Train the SVM model using grid search with cross-validation
svm_model = SVC()
svm_gs = GridSearchCV(svm_model, param_grid, cv=5, n_jobs=-1, verbose=1)
svm_gs.fit(X_train, y_train)

# Save the SVM model
joblib.dump(svm_gs.best_estimator_, 'svm_model.pkl')

# Predict on the test set using the SVM model
svm_y_pred = svm_gs.predict(X_test)

# Calculate the accuracy of the SVM model
svm_accuracy = accuracy_score(y_test, svm_y_pred)

print('Best parameters:', svm_gs.best_params_)
print('Best validation accuracy: {:.2f}%'.format(svm_gs.best_score_ * 100))
print('SVM accuracy:', svm_accuracy)

Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Best parameters: {'C': 10, 'coef0': 0.0, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Best validation accuracy: 94.83%
SVM accuracy: 0.9601990049751243


In [ ]:
from sklearn import svm

# Define the SVM model with the updated best hyperparameters
svm_model = svm.SVC(C=10, kernel='rbf', degree=2, gamma='scale', coef0=0.0)

# Train the model on your training data
svm_model.fit(X_train, y_train)

# Make predictions on your test data
y_pred = svm_model.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("SVM accuracy:", accuracy)

# Save the trained model to disk as a pkl file
with open('best_hyp_svm_mode_multiplefeature_matrixclassification.pkl', 'wb') as f:
    pickle.dump(svm_model, f)


SVM accuracy: 0.9601990049751243


# Knn by finding best neighbour

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import pickle

# Define the parameter grid for the number of neighbors
param_grid = {'n_neighbors': [1, 3, 5, 7, 9]}

# Define the KNN model
knn_model = KNeighborsClassifier()

# Perform grid search cross-validation to find the best number of neighbors
grid_search = GridSearchCV(knn_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best parameters:", grid_search.best_params_)

# Train the KNN model on your training data using the best number of neighbors
knn_model = KNeighborsClassifier(n_neighbors=grid_search.best_params_['n_neighbors'])
knn_model.fit(X_train, y_train)

# Make predictions on your test data
y_pred = knn_model.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("KNN accuracy:", accuracy)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Save the trained model to disk as a pkl file
with open('best_hyp_knn_model_multiplefeature_matrix_classification.pkl', 'wb') as f:
    pickle.dump(knn_model, f)


Best parameters: {'n_neighbors': 1}
KNN accuracy: 0.9402985074626866
Classification Report:
              precision    recall  f1-score   support

         0.0       0.92      0.96      0.94       203
         1.0       0.96      0.92      0.94       199

    accuracy                           0.94       402
   macro avg       0.94      0.94      0.94       402
weighted avg       0.94      0.94      0.94       402



In [ ]:
# linear_svm_model_singlefeature_matrixclassfifcation.pkl -
# cubic_svm_model_singlefeature_matrixclassfifcation.pkl -
# best_hyp_svm_model_singlefeature_matrixclassfifcation.pkl -
# knn_model_singlefeature_matrixclassfifcation.pkl -

# rbf_svm_model_multiplefeature_matrix_classification.pkl -
# cubic_svm_model_multiplefeature_matrix_classification -
# knn_model_multiplefeature_matrix_classification.pkl -
# best_hyp_svm_mode_multiplefeature_matrixclassification.pkl -
# best_hyp_knn_model_multiplefeature_matrix_classification.pkl -